In [2]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase,Predictor
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy

from scipy import stats


In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [3]:
from scipy.stats import pearsonr

# Parameters

In [4]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Entinostat'
na = 1

# Import data

In [5]:
data_r=pd.read_csv('data/GDSC2_expression.csv',index_col=0)
label_r=pd.read_csv('data/GDSC2_label_9drugs.csv',index_col=0)

In [6]:
label_r=label_r.fillna(na)

In [7]:
hvg,adata = ut.highly_variable_genes(data_r)

In [8]:
selected_idx = label_r.loc[:,select_drug]!=na

In [9]:
data_r.columns = adata.var_names

In [10]:
hvg.sum()

5116

# Your is gene-cell, mine is cell-gene

In [11]:
#data = data_r.loc[selected_idx,:]
data = data_r.loc[selected_idx,hvg]

In [12]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()
lbscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = lbscaler.fit_transform(label.values.reshape(-1,1))
#label = label.values.reshape(-1,1)

In [13]:
print(np.std(data))
print(np.mean(data))

0.25270809454700544
0.24264587007274258


In [14]:
data.mean(axis=0)


array([0.4758466 , 0.0788965 , 0.20600574, ..., 0.12230791, 0.0555018 ,
       0.40267655])

In [15]:
print(data.max())
print(data.min())

1.0000000000000002
0.0


In [16]:
data.shape

(732, 5116)

In [17]:
label_r.shape

(804, 9)

# Split test train

In [18]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [19]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(732, 5116)
(732, 1)
(468, 5116) (468, 1)
(147, 5116) (147, 1)


In [20]:
print(X_train.max())
print(X_train.min())

1.0000000000000002
0.0


# AE MODEL

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [22]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)


Y_trainTensor = torch.FloatTensor(Y_train).to(device)
Y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [23]:
dataloader = X_trainDataLoader

In [24]:
#dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

In [25]:
X_trainDataLoader.dataset.tensors[0].shape[0]

468

# The model

In [28]:
model = models.PretrainedPredictor(input_dim=5116,latent_dim=512,hidden_dims=[2048,1024], 
                            hidden_dims_predictor=[256,128],drop_out=0.6,
                            pretrained_weights='saved/models/GDSCnew_ae.pkl',freezed=True)

Layer weight is freezed: torch.Size([2048, 5116])
Layer weight is freezed: torch.Size([2048])
Layer weight is freezed: torch.Size([2048])
Layer weight is freezed: torch.Size([2048])
Layer weight is freezed: torch.Size([1024, 2048])
Layer weight is freezed: torch.Size([1024])
Layer weight is freezed: torch.Size([1024])
Layer weight is freezed: torch.Size([1024])
Layer weight is freezed: torch.Size([512, 1024])
Layer weight is freezed: torch.Size([512])


In [29]:
print(model)

PretrainedPredictor(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=5116, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.6, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2048, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.6, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=1024, out_features=512, bias=True)
  (predictor): Predictor(
    (predictor): Sequential(
      (0): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout(p=0.3, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentu

In [30]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [31]:
# Load data
# data type conversion

# y_trainTensor = torch.FloatTensor(Y_train).to(device)
# y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=200, shuffle=True)
validDataLoader_p = DataLoader(dataset=validreducedDataset, batch_size=200, shuffle=True)

In [32]:
dataloaders_train = {'train':trainDataLoader_p,'val':validDataLoader_p}

In [ ]:
model,report = ut.train_predictor_model(model,dataloaders_train,
                                        optimizer,loss_function,epochs,exp_lr_scheduler,save_path="saved/models/pre_pre_model.pkl")

Epoch 0/499
----------
train Loss: 0.00078376. Learning rate = 0.01
val Loss: 0.00200579. Learning rate = 0.01
Epoch 1/499
----------
train Loss: 0.00080389. Learning rate = 0.01
val Loss: 0.00130031. Learning rate = 0.01
Epoch 2/499
----------
train Loss: 0.00046627. Learning rate = 0.01
val Loss: 0.00096660. Learning rate = 0.01
Epoch 3/499
----------
train Loss: 0.00043200. Learning rate = 0.01
val Loss: 0.00070521. Learning rate = 0.01
Epoch 4/499
----------
train Loss: 0.00037301. Learning rate = 0.01
val Loss: 0.00055874. Learning rate = 0.01
Epoch 5/499
----------
train Loss: 0.00032100. Learning rate = 0.01
val Loss: 0.00054585. Learning rate = 0.01
Epoch 6/499
----------
train Loss: 0.00027005. Learning rate = 0.01
val Loss: 0.00030090. Learning rate = 0.01
Epoch 7/499
----------
train Loss: 0.00022496. Learning rate = 0.01
val Loss: 0.00023975. Learning rate = 0.01
Epoch 8/499
----------
train Loss: 0.00019505. Learning rate = 0.01
val Loss: 0.00027103. Learning rate = 0.01
E

Epoch 74/499
----------
train Loss: 0.00017531. Learning rate = 0.0001
val Loss: 0.00022174. Learning rate = 0.0001
Epoch 75/499
----------
train Loss: 0.00015900. Learning rate = 0.0001
val Loss: 0.00022174. Learning rate = 0.0001
Epoch 76/499
----------
train Loss: 0.00016883. Learning rate = 1e-05
val Loss: 0.00022172. Learning rate = 1e-05
Epoch 77/499
----------
train Loss: 0.00017382. Learning rate = 1e-05
val Loss: 0.00022198. Learning rate = 1e-05
Epoch 78/499
----------
train Loss: 0.00016870. Learning rate = 1e-05
val Loss: 0.00022188. Learning rate = 1e-05
Epoch 79/499
----------
train Loss: 0.00016560. Learning rate = 1e-05
val Loss: 0.00022186. Learning rate = 1e-05
Epoch 80/499
----------
train Loss: 0.00016731. Learning rate = 1e-05
val Loss: 0.00022168. Learning rate = 1e-05
Epoch 81/499
----------
train Loss: 0.00015755. Learning rate = 1e-05
val Loss: 0.00022135. Learning rate = 1e-05
Epoch 82/499
----------
train Loss: 0.00014902. Learning rate = 1e-05
val Loss: 0.00

Epoch 134/499
----------
train Loss: 0.00015926. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022046. Learning rate = 1.0000000000000004e-08
Epoch 135/499
----------
train Loss: 0.00015740. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022053. Learning rate = 1.0000000000000004e-08
Epoch 136/499
----------
train Loss: 0.00015968. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022071. Learning rate = 1.0000000000000004e-08
Epoch 137/499
----------
train Loss: 0.00016998. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022122. Learning rate = 1.0000000000000004e-08
Epoch 138/499
----------
train Loss: 0.00015964. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022096. Learning rate = 1.0000000000000004e-08
Epoch 139/499
----------
train Loss: 0.00015699. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022086. Learning rate = 1.0000000000000004e-08
Epoch 140/499
----------
train Loss: 0.00015537. Learning rate = 1.0000000000000004e-08
val Loss: 0.000221

Epoch 190/499
----------
train Loss: 0.00016126. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022112. Learning rate = 1.0000000000000004e-08
Epoch 191/499
----------
train Loss: 0.00016941. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022097. Learning rate = 1.0000000000000004e-08
Epoch 192/499
----------
train Loss: 0.00015673. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022127. Learning rate = 1.0000000000000004e-08
Epoch 193/499
----------
train Loss: 0.00016371. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022115. Learning rate = 1.0000000000000004e-08
Epoch 194/499
----------
train Loss: 0.00015847. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022137. Learning rate = 1.0000000000000004e-08
Epoch 195/499
----------
train Loss: 0.00015963. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022098. Learning rate = 1.0000000000000004e-08
Epoch 196/499
----------
train Loss: 0.00015129. Learning rate = 1.0000000000000004e-08
val Loss: 0.000220

Epoch 246/499
----------
train Loss: 0.00016595. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022111. Learning rate = 1.0000000000000004e-08
Epoch 247/499
----------
train Loss: 0.00015698. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022104. Learning rate = 1.0000000000000004e-08
Epoch 248/499
----------
train Loss: 0.00014997. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022100. Learning rate = 1.0000000000000004e-08
Epoch 249/499
----------
train Loss: 0.00015336. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022110. Learning rate = 1.0000000000000004e-08
Epoch 250/499
----------
train Loss: 0.00016564. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022134. Learning rate = 1.0000000000000004e-08
Epoch 251/499
----------
train Loss: 0.00015205. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022058. Learning rate = 1.0000000000000004e-08
Epoch 252/499
----------
train Loss: 0.00015803. Learning rate = 1.0000000000000004e-08
val Loss: 0.000220

Epoch 301/499
----------
train Loss: 0.00015589. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022133. Learning rate = 1.0000000000000004e-08
Epoch 302/499
----------
train Loss: 0.00016052. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022106. Learning rate = 1.0000000000000004e-08
Epoch 303/499
----------
train Loss: 0.00016632. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022113. Learning rate = 1.0000000000000004e-08
Epoch 304/499
----------
train Loss: 0.00016633. Learning rate = 1.0000000000000004e-08
val Loss: 0.00022140. Learning rate = 1.0000000000000004e-08
Epoch 305/499
----------
train Loss: 0.00016716. Learning rate = 1.0000000000000004e-08


In [ ]:
list(model.children())[:2]

In [ ]:
model(X_testTensor)

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
dl_result = model(X_testTensor).detach().cpu().numpy()

In [ ]:
r2_score(dl_result,Y_test)

In [ ]:
pearsonr(dl_result.flatten(),Y_test.flatten())

In [ ]:
mean_squared_error(dl_result,Y_test)